In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as pltim
from PIL import Image as im
import numpy as np
import os 
import tensorflow as tf

In [2]:
#reading images
all_photos =  np.ndarray([255, 256, 256, 4])#85*3, photo dims
all_photos_names =  []
print(all_photos.shape)

#reading Lines
for i, filename in enumerate(os.listdir("Line_TS")):
    all_photos[i] = pltim.imread("Line_TS/" + filename, format="bmp")
    all_photos_names.append(filename)

#reading Ellipse
for i, filename in enumerate(os.listdir("Ellipse_TS")):
    all_photos[i+85] = pltim.imread("Ellipse_TS/" + filename, format="bmp")
    all_photos_names.append(filename)
    
#reading Lines
for i, filename in enumerate(os.listdir("Diamond_TS")):
    all_photos[i+170] = pltim.imread("Diamond_TS/" + filename, format="bmp")
    all_photos_names.append(filename)

print(len(all_photos_names))
img2 = im.open("Line_TS/" + all_photos_names[0])
img2.show()

img2 = im.open("Ellipse_TS/" + all_photos_names[85])
img2.show()

img2 = im.open("Diamond_TS/" + all_photos_names[170])
img2.show()    

(255, 256, 256, 4)
255


In [3]:
#labels(ground_truth) as one hot key
all_gt =  np.zeros([255, 3])

for i in range(255):
    if i > 169:
        all_gt[i][2] = 1
    elif i > 84:
        all_gt[i][1] = 1
    else:
        all_gt[i][0] = 1

# print(all_gt[0:])

In [4]:
#Training data(210) 
X_tr = np.ndarray([210,256, 256, 4])
y_tr = np.ndarray([210,3])

X_tr[0:70] = all_photos[0:70] #first 70 lines
X_tr[70:140] = all_photos[85:155] #first 70 ellipses
X_tr[140:210] = all_photos[170:240] #first 70 diamonds

y_tr[0:70] = all_gt[0:70]
y_tr[70:140] = all_gt[85:155]
y_tr[140:210] = all_gt[170:240]

print(X_tr.shape)
print(y_tr.shape)
# print(X_tr[0:10])

(210, 256, 256, 4)
(210, 3)


In [5]:
# Validation data(45)
X_v = np.ndarray([45, 256, 256, 4])
y_v = np.ndarray([45,3])

X_v[0:15] = all_photos[70:85]
X_v[15:30] = all_photos[155:170]
X_v[30:45] = all_photos[240:255]

y_v[0:15] = all_gt[70:85]
y_v[15:30] = all_gt[155:170]
y_v[30:45] = all_gt[240:255]

print(y_v.shape)
# print(y_v[0:])

(45, 3)


In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

keep_prob = tf.placeholder(tf.float32)

In [8]:
#Inputs
x = tf.placeholder(tf.float32, shape=[None, 262144])
x_image = tf.reshape(x, [-1, 256, 256, 4])

#Gorund truth
y_ = tf.placeholder(tf.float32, [None, 3])

In [10]:
#--Layer1
W_conv1 = weight_variable([5, 5, 4, 32])
b_conv1 = bias_variable([32])
batch_norm_conv1 = tf.contrib.layers.batch_norm(conv2d(x_image, W_conv1) + b_conv1)
h_conv1 = tf.nn.relu(batch_norm_conv1)
h_conv1_drop = tf.nn.dropout(h_conv1, 0.5)
#h_pool1 = max_pool_2x2(h_conv1)

#--Layer2
W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])
batch_norm_conv2 = tf.contrib.layers.batch_norm( conv2d(h_conv1_drop, W_conv2) + b_conv2 )
h_conv2 = tf.nn.relu( batch_norm_conv2)
h_conv2_drop = tf.nn.dropout(h_conv2, 0.8)
print (h_conv2_drop.shape)

#--fc1
W_fc1 = weight_variable([256 * 256 * 64, 256]) #256*256*4*64?!
b_fc1 = bias_variable([256])
h_conv2_flat = tf.reshape(h_conv2_drop, [-1, 256*256*64]) #256*256*4*64?!
batch_norm_fc1 = tf.contrib.layers.batch_norm( tf.matmul(h_conv2_flat, W_fc1) + b_fc1 )
h_fc1 = tf.nn.relu(batch_norm_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, 0.6)
print (h_conv2_flat.shape)
print(W_fc1)

#--fc3
W_fc2 = weight_variable([256, 64])
b_fc2 = bias_variable([64])
batch_norm_fc2 = tf.contrib.layers.batch_norm( tf.matmul(h_fc1_drop, W_fc2)  + b_fc2 )
h_fc2 = tf.nn.relu(batch_norm_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, 0.9)

#Readout layer
W_fc3 = weight_variable([64, 3])
b_fc3 = bias_variable([3])

y_conv = tf.matmul(h_fc2_drop, W_fc3) + b_fc3

(?, 256, 256, 64)
(?, 4194304)
Tensor("Variable_14/read:0", shape=(4194304, 256), dtype=float32)


In [11]:
#Evaluate error
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
#Update params
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#compare with the ground truth
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
from time import time

tic = time()

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(4):
                
        if i % 2 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: X_tr, y_: y_tr})
            print('step %d, training accuracy %g' % (i, train_accuracy))
            
        #train_step.run(feed_dict={x: X_tr, y_: y_tr, keep_prob: 0.5}) how to not to use dropout in test?
        train_step.run(feed_dict={x: X_tr, y_: y_tr})
    
toc = time()

print('take time: %s' %((toc - tic)/60))

In [ ]:
#Evaluating
print('test accuracy %g' % accuracy.eval(feed_dict={x: X_v, y_: y_v}))
    #writer = tf.summary.FileWriter('./Model_Summary' , sess.graph)

In [ ]:
#Where are my weights now?! 
#batches
#check_ponits(every specific time ?!) mkanha feen sa7?!
#Save, Restore
#Analysis (model summary and tensorboard)
#Converting to cnn

In [ ]:
#Saving and creating checkpoints
saver = tf.train.Saver()
save_path = saver.save(sess, "chkpts/model_1n.ckpt")

In [ ]:
#Restoring variables
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, "chkpts/model_1n.ckpt")
    print("Model restored.")

In [ ]:
# momken ba2a ba3d ama a-restore el model a3mel test ezai ?!
print('test accuracy %g' % accuracy.eval(feed_dict={x: X_v, y_: y_v}))

In [ ]:
from IPython.display import Audio
sound_file = './sound/finish.mp3'
Audio(url=sound_file, autoplay=True)